In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00


#### Loading audio dataset

In [2]:
from datasets import load_dataset

# This dataset is a collection of different sounds of 5 seconds
dataset = load_dataset("ashraq/esc50")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [3]:
# audio = dataset[3]
audio = dataset['train'][100]

In [4]:
audio

{'filename': '1-208757-B-2.wav',
 'fold': 1,
 'target': 2,
 'category': 'pig',
 'esc10': False,
 'src_file': 208757,
 'take': 'B',
 'audio': {'path': None,
  'array': array([ 0.07342529,  0.04425049,  0.01834106, ..., -0.03030396,
         -0.02676392, -0.0256958 ]),
  'sampling_rate': 44100}}

#### Playing audio

In [26]:
from IPython.display import Audio as IPythonAudio
IPythonAudio(audio['audio']['array'], rate=audio['audio']['sampling_rate'])

In [5]:
from transformers import pipeline

In [6]:
classifier = pipeline(
    'zero-shot-audio-classification',
    model = 'laion/clap-htsat-unfused'
)

config.json:   0%|          | 0.00/5.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/615M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

In [7]:
audio['audio']['array']

array([ 0.07342529,  0.04425049,  0.01834106, ..., -0.03030396,
       -0.02676392, -0.0256958 ])

#### Performing classification

In [8]:
classifier(audio['audio']['array'], candidate_labels = ['Sound of dog', 'sound of human eating'])

[{'score': 0.8665752410888672, 'label': 'sound of human eating'},
 {'score': 0.13342471420764923, 'label': 'Sound of dog'}]

#### Mostly sampling_rate of audio to be classified and model does not match which produces wrong results, that's why we need to make same rate of audio and model

In [9]:
classifier.feature_extractor.sampling_rate

48000

In [10]:
dataset['train'][90]

{'filename': '1-19872-A-36.wav',
 'fold': 1,
 'target': 36,
 'category': 'vacuum_cleaner',
 'esc10': False,
 'src_file': 19872,
 'take': 'A',
 'audio': {'path': None,
  'array': array([-0.01992798, -0.02520752, -0.04702759, ..., -0.0519104 ,
         -0.05578613, -0.04269409]),
  'sampling_rate': 44100}}

#### Converting sampling rate of dataset equal to model

In [20]:
from datasets import Audio
dataset = dataset.cast_column(
    'audio',
    Audio(sampling_rate= 48000)
)

In [21]:
dataset['train'][90]

{'filename': '1-19872-A-36.wav',
 'fold': 1,
 'target': 36,
 'category': 'vacuum_cleaner',
 'esc10': False,
 'src_file': 19872,
 'take': 'A',
 'audio': {'path': None,
  'array': array([-0.01862913, -0.02456462, -0.04301245, ..., -0.05695524,
         -0.03873681,  0.        ]),
  'sampling_rate': 48000}}

- We can see the difference between the performance of model because of changed sampling rate

In [22]:
audio1 = dataset['train'][100]
classifier(audio['audio']['array'], candidate_labels = ['sound of dog', 'sound of human eating'])

[{'score': 0.9008726477622986, 'label': 'sound of human eating'},
 {'score': 0.09912732243537903, 'label': 'sound of dog'}]

In [23]:
audio2 = dataset['train'][50]

In [28]:
classifier(audio2['audio']['array'], candidate_labels=['sound of human', 'sound of machine', 'sound of water'])

[{'score': 0.9988644123077393, 'label': 'sound of water'},
 {'score': 0.000637793680652976, 'label': 'sound of human'},
 {'score': 0.0004977776552550495, 'label': 'sound of machine'}]

In [27]:
IPythonAudio(audio2['audio']['array'], rate=audio2['audio']['sampling_rate'])

#### Text Classification
- As we are performing, audio classification in the same way we can perform text classification by simply providing text

In [ ]:
from transformers import pipeline

In [ ]:
zero_shot_classifier = pipeline(
  'zero-shot-classification',
  model = 'facebook/bart-large-mnli'
)

In [ ]:
text_to_classify = '''
I want to go for higher studies and have an amazing time
'''

In [ ]:
candidate_labels = ['travel', 'study', 'walking']

In [ ]:
zero_shot_classifier(text_to_classify, candidate_labels)

{'sequence': '\nI want to go for higher studies and have an amazing time\n',
 'labels': ['study', 'travel', 'walking'],
 'scores': [0.8426235914230347, 0.12378431856632233, 0.03359212353825569]}